# <font color='black'> НИС: регрессионный анализ, 2025 </font>
# <font color='black'> О контрольных переменных и переменных-следствиях (коллайдерах) </font>

На этом занятии мы продолжим работать с данными из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Авторы изучают взаимосвязь уровня коррупции и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Непрерывная шкала от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения. World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from scipy.stats import norm

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set(style = "white", palette='deep')

Откроем массив данных для репликации результатов исследования - lab1.dta.

In [ ]:
lab2 = pd.read_stata('lab1.dta')
lab2 = lab2.dropna()

Оценим для начала регрессионную модель без переменных взаимодействия. Проинтерпретируйте все оценки коэффициентов модели m1. Как изменилась оценка коэффициента при dem по сравнению с соответствующей оценкой в модели m0?

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem + fp + loggdppc + britcol + stab", data = lab2).fit(cov_type = "HC3")
print(m1.summary())

Альтернатива: Мы можем получить оценку коэффициента при любом предикторе в такой модели, воспользуясь теоремой Фриша-Во-Ловелла (the Frisch-Waugh-Lovell theorem). К примеру, для того, чтобы получить оценку коэффициента при переменной dem, а именно, оценить, как связаны dem и cpi, нужно очистить вариацию этих переменных от других предикторов (а именно, от fp, loggdppc, britcol и stab, с другой).

1) Для этого мы оцениваем регрессию dem на fp, loggdppc, britcol и stab, сохраняем остатки - то есть, получаем очищенный показатель dem.

2) Далее аналогично оцениваем регрессию cpi на fp, loggdppc, britcol и stab, сохраняем остатки - то есть, получаем очищенный показатель cpi.

3) После этого достаточно будет оценить регрессию очищенного cpi на очищенный dem и убедиться, что мы получили тот же самый коэффициент при dem, что и в исходной модели с контрольными переменными.

In [ ]:
m1_1 = smf.ols(formula = "dem ~ fp + loggdppc + britcol + stab", data = lab2).fit()
resid_data = pd.DataFrame()
resid_data["res1"] = m1_1.resid

In [ ]:
m1_2 = smf.ols(formula = "cpi ~ fp + loggdppc + britcol + stab", data = lab2).fit()
resid_data["res2"] = m1_2.resid

In [ ]:
m1_3 = smf.ols(formula = "res2 ~ res1", data = resid_data).fit(cov_type = "HC3")
print(m1_3.summary())

Для того, чтобы лучше понять, что скрывается за процессом очищения вариации, представим себе следующее. Мы включаем в модель категориальную контрольную переменную. Далее реализуем следующие шаги:

1) разделим исходный массив на подгруппы в зависимости от количества категорий в контрольной переменной

2) оценим регрессионную модель $y$ на $x$ для каждого подмассива, итого получим J оценок коэффициентов наклона при $x$, где J - это количество категорий контрольной переменной   

3) взвесим результаты: суммируем взвешенные оценки коэффициентов, в качестве веса будет выступать доля подвыборки в общем массиве $∑w_j\hat{\beta}_j$

Таким образом, очищение вариации равносильно тому, как если бы мы исследовали взаимосвязь $x$ и $y$ при фиксированном значении контрольной переменной

Рассмотрим для иллюстрации исходную модель. При этом сфокусируемся на разделении выборки по бинарной контрольной переменной britcol

Итак, всего у нас 170 наблюдений

In [ ]:
len(lab2)

Из них 38 стран - бывшие британские колонии, оставшиеся 132 страны - страны, которые не являются бывшими британскими колониями

In [ ]:
print(f"Страны, которые не являются бывшими британскими колониями: {len(lab2[lab2['britcol'] == 0])} стран")
print(f"Бывшие британские колонии: {len(lab2[lab2['britcol'] == 1])} стран")

Оценим регрессионную модель на соответствующих подвыборках:

In [ ]:
m_group0 = smf.ols(formula='cpi ~ dem + fp + loggdppc + stab',
                      data=lab2[lab2['britcol'] == 0]).fit()

print(m_group0.summary())

In [ ]:
m_group1 = smf.ols(formula='cpi ~ dem + fp + loggdppc + stab',
                      data=lab2[lab2['britcol'] == 1]).fit()

print(m_group1.summary())

Сравним результаты в полной модели с учетом контрольной переменной britcol и с учетом деления на подвыборки:

In [ ]:
X = ['dem', 'fp', 'loggdppc', 'stab']
comparison = []

for x in X:
        comparison.append({
            'variable': x,
            'group0': m_group0.params[x],
            'group1': m_group1.params[x],
            'total': m1.params[x],
            'weight_group0': len(lab2[lab2['britcol'] == 0]) / len(lab2),
            'weight_group1': len(lab2[lab2['britcol'] == 1]) / len(lab2)
        })

comparison = pd.DataFrame(comparison)

comparison

Взвесим результаты на долю наблюдений.

In [ ]:
comparison['weighted_coef'] = comparison['group0']*comparison['weight_group0'] + comparison['group1']*comparison['weight_group1']

comparison

Полученные результаты взвешивания показывают близкие результаты к исходным результатам оценивания модели на всей выборке.

Несколько сложнее обстоят дела с непрерывной контрольной переменной. Размер подвыборки, с одной стороны, должен быть не слишком маленьким для оценивания регрессионной модели с заданным количеством параметров. С другой стороны, создавать слишком большие подвыборки также нецелесообразно, так как можно упустить различия во взаимосвязи, проявляющиеся в разных подмассивах, и в результате получить плохое приближение к оценкам модели по всей выборке. В связи с этим поделим массив на подвыборки из расчета, чтобы в подвыборке было хотя бы 30 наблюдений (с учетом нагруженности спецификации модели следует корректировать размер подвыборки):     

In [ ]:
lab2['stab_quartile'] = pd.qcut(lab2['stab'], q=4, labels=False)

X = ['dem', 'fp', 'loggdppc', 'britcol']
comparison = []

for x in X:
    coefs_by_quartile = []
    weights_by_quartile = []

    for quartile in sorted(lab2['stab_quartile'].unique()):
        quartile_data = lab2[lab2['stab_quartile'] == quartile]

        if len(quartile_data) > 30:
            model = smf.ols('cpi ~ dem + fp + loggdppc + britcol', data=quartile_data).fit()
            coefs_by_quartile.append(model.params[x])
            weights_by_quartile.append(len(quartile_data) / len(lab2))

    weighted_coef = np.average(coefs_by_quartile, weights=weights_by_quartile)

    comparison.append({
        'variable': x,
        'total': m1.params[x],
        'weighted_coef': weighted_coef
    })

comparison = pd.DataFrame(comparison)
comparison

При этом переменные-"общие следствия" (collider variables) являются вредными контрольными переменными. В исходных данных мы можем наблюдать отсутствие связи либо слабую связь. Однако при включении переменной-коллайдера может возникнуть ложная связь: к примеру, $P(Y=1 | X=1 ∩ C=1) ≠ P(Y=1 | X=0 ∩ C=1)$. То есть, при фиксированном значении коллайдера возникает связь, которой изначально могло и не быть в исходном массиве

Сгенерим данные для того, чтобы проиллюстрировать смещение в оценках коэффициентов в результате включения неподходящей контрольной переменной:

In [ ]:
np.random.seed(42)

dem_r = norm.rvs(5, 1.5, 1000)

cpi_r = 10 - 1.2*dem_r + norm.rvs(0, 1, 1000)

educ_r = 5 + 0.8*dem_r - 0.7 * cpi_r + norm.rvs(0, 0.5, 1000)

df = pd.DataFrame({
    'cpi_r': cpi_r,
    'dem_r': dem_r,
    'educ_r': educ_r
})

df.head()

In [ ]:
round(df['dem_r'].corr(df['cpi_r']), 3)

In [ ]:
plt.figure(figsize=(8, 4))

scatter = sns.regplot(data=df, x="dem_r", y="cpi_r")

plt.title("Взаимосвязь индексов демократии и коррупции",
          fontsize=16, fontweight='bold')
plt.xlabel("Индекс демократии", fontsize=12, fontweight='bold')
plt.ylabel("Индекс восприятия коррупции", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
m_r1 = smf.ols('cpi_r ~ dem_r', data=df).fit(cov_type = "HC3")
print(m_r1.summary())


In [ ]:
m_educ_r = smf.ols('cpi_r ~ dem_r + educ_r', data=df).fit(cov_type = "HC3")

print(m_educ_r.summary())

In [ ]:
df['educ_levels'] = pd.qcut(df['educ_r'], q=30, labels=False)

X = ['dem_r']
comparison = []

for x in X:
    coefs_by_quartile = []
    weights_by_quartile = []

    for quartile in sorted(df['educ_levels'].unique()):
        quartile_data = df[df['educ_levels'] == quartile]

        if len(quartile_data) > 30:
            model = smf.ols('cpi_r ~ dem_r', data=quartile_data).fit()
            coefs_by_quartile.append(model.params[x])
            weights_by_quartile.append(len(quartile_data) / len(df))

    weighted_coef = np.average(coefs_by_quartile, weights=weights_by_quartile)

    comparison.append({
        'variable': x,
        'total': m_r1.params[x],
        'weighted_coef': weighted_coef
    })

comparison = pd.DataFrame(comparison)
comparison